In [1]:
# import pandas as pd
import chess.pgn
import numpy as np

In [2]:
# # Not used for now
# # Read data
# games = pd.read_csv("./2020_2000ELO_Standard.pgn", sep='delimiter', header=None)
# # Filter just game moves
# games = games[games[0].str.startswith("1.")]
# # Rename column 0 to pgn
# games.rename(columns={0:'pgn'}, inplace=True)
# # Drop index
# games.reset_index(drop=True)

In [3]:
# Import all pgn games with python chess
pgn = open("./2020_2000ELO_Standard.pgn")

games = []

while True:
    game = chess.pgn.read_game(pgn)
    if game is None:
        break  # end of file
    games.append(game)

In [4]:
def fen_to_matrix(fen):
    # TODO: Flip when players are reversed
    print(fen)
    board = []
    pieces = ['K', 'Q', 'R', 'B', 'N', 'P', 'k', 'q', 'r', 'b', 'n', 'p']
    # Take just the board
    fen = fen.split(' ')[0]
    # Split by rows
    rows = fen.split('/')
    # For each piece mark the corresponding matrix location with a 1
    for piece in pieces:
        piece_map = np.zeros((8,8), dtype='int')
        for i, row in enumerate(rows):
            j = 0
            for char in row:
                if char is piece:
                    piece_map[i][j] = 1
                    j += 1
                elif char.isdigit():
                    j += int(char)
                else:
                    j += 1
                    
        board.append(piece_map)
    return np.array(board)

In [90]:
board = fen_to_matrix(games[0].board().fen())
print(np.shape(board))
# print(board)

rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
(12, 8, 8)


In [93]:
def move_to_coords(move):
    coords = [-1, -1]
    # Coords are the opposite order of UCI to maintain [Row, Column] order
    # Letter component
    coords[1] = ord(move[0]) - 97
    # Number component (subtract from 7 to maintain board orientation)
    coords[0] = 7 - (int(move[1]) - 1)
    # Check for invalid coordinates
    if any(i < 0 for i in coords):
        print("ERROR - INVALID COORDINATES", coords)
    return coords

# Take the difference of cordinates and return if diagonal, more up knight move, or more sideways knight move
def diff_to_subdirection(diff):
    # Diagonal
    if diff[0] == diff[1]:
        return 0
    # More up than sideways
    elif abs(diff[0]) > abs(diff[1]):
        return 1
    # More sideways than up
    else:
        return 2

# Take the difference of coordinates and calculate direction of move
def diff_to_direction(diff):
    # North moves
    if diff[0] < 0:
        # True north
        if diff[1] == 0:
            return "N"
        # North east
        if diff[1] > 0:
            directions = ["NE", "NNE", "NEE"]
            return directions[diff_to_subdirection(diff)]
        # North west
        if diff[1] < 0:
            directions = ["NW", "NNW", "NWW"]
            return directions[diff_to_subdirection(diff)]
    # South moves
    elif diff[0] > 0:
        # True south
        if diff[1] == 0:
            return "S"
        # South east
        if diff[1] > 0:
            directions = ["SE", "SSE", "SEE"]
            return directions[diff_to_subdirection(diff)]
        # South west
        if diff[1] < 0:
            directions = ["SW", "SSW", "SWW"]
            return directions[diff_to_subdirection(diff)]
    # Sideways moves (East/West)
    elif diff[0] == 0:
        if diff[1] > 0:
            return "E"
        elif diff[1] < 0:
            return "W"
        else:
            print("ERROR - Invalid Move")
    else:
        print("ERROR - Invalid Move")

# Take chess UCI move as input and return alphazero move matrix for that move (8, 8, 73)
def uci_to_move_matrix(uci):
    move_matrix = np.zeros((73,8,8), dtype='int')
    
    if uci == '0000':
        print("ERROR - NULL MOVE:", uci)
        
    # There is no promotion
    if len(uci) == 4:
        start = move_to_coords(uci[:2])
        end = move_to_coords(uci[2:])
        # Calculate difference of coords to work out what move it is (out of the 73 (64 for non-promition) in the alphazero paper)
        diff = np.subtract(end, start)
        # Dictionary to hold different move codes
        number_moves = 7
        moves = {
            # Queen moves
            "N": 0*number_moves,
            "NE": 1*number_moves,
            "E": 2*number_moves,
            "SE": 3*number_moves,
            "S": 4*number_moves,
            "SW": 5*number_moves,
            "W": 6*number_moves,
            "NW": 7*number_moves,
            # Knight moves
            "NNE": 56,
            "NEE": 57,
            "SEE": 58,
            "SSE": 59,
            "SSW": 60,
            "SWW": 61,
            "NWW": 62,
            "NNW": 63
        }
        move_direction = diff_to_direction(diff)
        # The matrix offset of a move (e.g. moving north 2 squares = 1)
        # See alphazero paper on representations for more info
        move_matrix_offset = moves[move_direction]
        # If queen, add on the number it moves
        if move_matrix_offset < 56:
            move_matrix_offset += np.amax(np.absolute(diff)) - 1
        # Update the move matrix with the move
        move_matrix[move_matrix_offset][start[0]][start[1]] = 1
        
    # There is a promotion
    elif len(uci) == 5:
        # Todo, convert into possible 8x8x9 (9 moves) matrix for each pawn promotion
        print(uci)
        
    else:
        print("ERROR - UNRECOGNISED UCI:", uci)
        
    return move_matrix
        

In [95]:
# List of initial moves in initial game state
board = games[0].board()
legal_moves = board.legal_moves
for move in legal_moves:
    uci_to_move_matrix(move.uci())

In [8]:
# Move through entire game to end state of game
board = games[0].board()
for move in games[0].mainline_moves():
    board.push(move)
print(board)
print(board.fen())

r . b . . . k .
. p . . b . P .
. . . . p r N .
. . . . . p Q B
. . . q . . . .
. P . . . n . .
. . . . . . P P
q . . . . . . K
r1b3k1/1p2b1P1/4prN1/5pQB/3q4/1P3n2/6PP/q6K w - - 2 36
